In [ ]:
from control import lqr
import numpy as np
from matplotlib import pyplot as plt
import os, re
import pandas as pd
from matplotlib import font_manager  #matplotlib中 中文设置模块

In [ ]:
## 输出高清图像
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import platform
# 图像显示中文的问题，需要判断系统是windows还是苹果的
import matplotlib
import platform
sys_platform = platform.platform().lower()
if "windows" in sys_platform:
    font = {
    "family": "Times New Roman"
    }
    matplotlib.rc("font", **font)
else:
    font = {
    "family": "Arial Unicode MS"
    }
    matplotlib.rc("font", **font)
rc = {"mathtext.fontset" : "stix",}

plt.rcParams.update(rc)

# 提取数据

## Libs

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df=pd.read_csv('./data/opensees_design_res.csv')
df.info()
df['ok'] = True
df.loc[df["drift_max"] > 1 / 200, ['ok']]=False
df.loc[df["a_max"] > 0.35 * 9.8, ['ok']]=False
df.sample(3)
df.info

In [ ]:
df["ok"].value_counts()

In [ ]:
df.describe()

# 分类问题

# Libs

In [ ]:
from sklearn.model_selection  import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
# 利用GridSearchCV选择最优参数
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.metrics import plot_roc_curve
from sklearn.linear_model import LogisticRegression

from mlxtend.classifier import StackingClassifier

from collections import Counter

## 特征过程

In [ ]:
# 去掉id,site_name,miu,zeta,kappa,gamma,alpha,
x_label = ["m1", "m2", "m3", "m4", "m5", "m6", "m7", "m8", "m9", "k1", "k2", "k3", "k4", "k5", "k6", "k7", "k8", "k9", "c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9", "sa1", "sa2", "sa3", "sa4", "sa5", "sa6", "sa7", "sa8", "sa9", "sa10", "sa11", "sa12", "sa13", "sa14", "sa15", "sa16", "sa17", "sa18", "sa19", "sa20", "sa21", "sa22", "sa23", "sa24", "sa25", "sa26", "sa27", "sa28", "sa29", "sa30", "sv1", "sv2", "sv3", "sv4", "sv5", "sv6", "sv7", "sv8", "sv9", "sv10", "sv11", "sv12", "sv13", "sv14", "sv15", "sv16", "sv17", "sv18", "sv19", "sv20", "sv21", "sv22", "sv23", "sv24", "sv25", "sv26", "sv27", "sv28", "sv29", "sv30", "sd1", "sd2", "sd3", "sd4", "sd5", "sd6", "sd7", "sd8", "sd9", "sd10", "sd11", "sd12", "sd13", "sd14", "sd15", "sd16", "sd17", "sd18", "sd19", "sd20", "sd21", "sd22", "sd23", "sd24", "sd25", "sd26", "sd27", "sd28", "sd29", "sd30", "sa_max", "sa_avg", "sv_max", "sv_avg", "sd_max", "sd_avg", "pga", "pgv", "pgd", "epa", "epv", "epd", "pa", "pv", "pd", "ic"]
y_label = ["ok"]
X = df.loc[:, x_label].values
y = df.loc[:, y_label].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)
# y_test = y_test.reshape([-1,])

In [ ]:
# shuffle_index = np.random.permutation(934)
# X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
new_X_train = X_train
new_X_test = X_test

# normalizer = Normalizer(copy=True, norm='l2').fit(new_X_train)
ss = StandardScaler().fit(new_X_train)
new_X_train = ss.transform(new_X_train)
new_X_test = ss.transform(new_X_test)

In [ ]:
new_X_test
# model = DecisionTreeClassifier()
# param = {'criterion': ['gini', 'entropy'], 'max_depth': [10, 20,30,40, 50, 60,70,80,90,100], 'min_samples_leaf': [2, 3, 5, 10], 'min_impurity_decrease': [0.1, 0.2, 0.5]}
# grid = GridSearchCV(model, param_grid=param, cv=5)
# grid.fit(new_X_train, y_train)
# print('最优分类器:', grid.best_estimator_)
# print('最优超参数：', grid.best_params_)
# print('最优分数:', grid.best_score_)


## 随机森林

In [ ]:
rfc = RandomForestClassifier(random_state=3)
print(rfc)
rfc.fit(new_X_train, y_train)
y_pre_rfc = rfc.predict(new_X_test)

print('正确标签：', y_test)
print('预测结果：', y_pre_rfc)

print('训练集分数：', rfc.score(new_X_train, y_train))
print('测试集分数：', rfc.score(new_X_test, y_test))
# 混淆矩阵
conf_mat = confusion_matrix(y_test, y_pre_rfc)
plt.matshow(conf_mat, cmap=plt.cm.Blues)
for (i, j), z in np.ndenumerate(conf_mat):
    plt.text(j, i, '{0:d}'.format(z), ha='center', va='center')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()
print('混淆矩阵：')
print(conf_mat)

print('混淆矩阵：')
print(conf_mat)


# 分类指标文本报告（精确率、召回率、F1值等）
print('分类指标报告：')
print(classification_report(y_test, y_pre_rfc))

# 特征重要性
print(rfc.feature_importances_)

# 画图展示训练结果
fig = plt.figure()
ax = fig.add_subplot(111)
f2 = ax.scatter(list(range(len(X_test))), y_pre_rfc, marker='o')
f1 = ax.scatter(list(range(len(X_test))), y_test, marker='*')
plt.legend(handles=[f1, f2], labels=['True', 'Prediction'])
plt.show()

In [ ]:
def plot_features_importances(feature_importances, feature_names):
    plt.figure(figsize=(5, 20))
    indices = np.argsort(feature_importances) # 下标排序
    indices_flip = indices[::-1] # 倒序
    names = []
    for f in range(len(feature_names)):
        names.append(feature_names[indices[f]])
        print("%2d) %-*s %f" % \
              (f + 1, 30, feature_names[indices_flip[f]], feature_importances[indices_flip[f]]))
    pos = np.arange(indices.shape[0])
    plt.barh(pos, feature_importances[indices], align="center")
    plt.ylim([pos[0] - 1, pos[-1] + 1])
    plt.ylabel("排序", fontname='SimSun', fontsize=10)
    plt.xlabel("相对重要性", fontname='SimSun', fontsize=10)
    plt.yticks(pos, names, fontsize=5)
    plt.xticks(fontsize=5)
    xlabel = feature_importances[indices]
    ylabel = pos
    for x1, y1 in zip(xlabel, ylabel):
        x1 = np.around(x1, decimals=3)
        plt.text(x1 + 0.0005, y1 - 0.25, '%.3f' % x1, fontsize=5)
    plt.show()

In [ ]:
plot_features_importances(rfc.feature_importances_, x_label)

 ## 神经网络

In [ ]:
mlpc =  MLPClassifier(hidden_layer_sizes=(100, 50, 25), max_iter=1000,learning_rate_init=0.001)
mlpc.fit(new_X_train, y_train)
y_pre_rfc = mlpc.predict(new_X_test)

print('正确标签：', y_test)
print('预测结果：', y_pre_rfc)

print('训练集分数：', rfc.score(new_X_train, y_train))
print('测试集分数：', rfc.score(new_X_test, y_test))
# 混淆矩阵
conf_mat = confusion_matrix(y_test, y_pre_rfc)
plt.matshow(conf_mat, cmap=plt.cm.Blues)
for (i, j), z in np.ndenumerate(conf_mat):
    plt.text(j, i, '{0:d}'.format(z), ha='center', va='center')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

print('混淆矩阵：')
print(conf_mat)


# 分类指标文本报告（精确率、召回率、F1值等）
print('分类指标报告：')
print(classification_report(y_test, y_pre_rfc))


# 画图展示训练结果
fig = plt.figure()
ax = fig.add_subplot(111)
f2 = ax.scatter(list(range(len(X_test))), y_pre_rfc, marker='o')
f1 = ax.scatter(list(range(len(X_test))), y_test, marker='*')
plt.legend(handles=[f1, f2], labels=['True', 'Prediction'])
plt.show()

## KNN

In [ ]:
knc = KNeighborsClassifier()
knc.fit(new_X_train, y_train)
y_pre_rfc = knc.predict(new_X_test)

print('正确标签：', y_test)
print('预测结果：', y_pre_rfc)

print('训练集分数：', rfc.score(new_X_train, y_train))
print('测试集分数：', rfc.score(new_X_test, y_test))
# 混淆矩阵
conf_mat = confusion_matrix(y_test, y_pre_rfc)
plt.matshow(conf_mat, cmap=plt.cm.Blues)
for (i, j), z in np.ndenumerate(conf_mat):
    plt.text(j, i, '{0:d}'.format(z), ha='center', va='center')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

print('混淆矩阵：')
print(conf_mat)


# 分类指标文本报告（精确率、召回率、F1值等）
print('分类指标报告：')
print(classification_report(y_test, y_pre_rfc))


# 画图展示训练结果
fig = plt.figure()
ax = fig.add_subplot(111)
f2 = ax.scatter(list(range(len(X_test))), y_pre_rfc, marker='o')
f1 = ax.scatter(list(range(len(X_test))), y_test, marker='*')
plt.legend(handles=[f1, f2], labels=['True', 'Prediction'])
plt.show()

# Stacking

In [ ]:
class AveragingModels(ClassifierMixin, BaseEstimator):
    def __init__(self, models):
        self.models = models

    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]

        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self

    #Now we do the predictions for cloned models and vote for result
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        res_len = predictions.shape[0]
        pred_res = np.empty((res_len,), np.bool_)
        for i in range(res_len):
            counter = Counter(predictions[i].tolist())
            pred_res[i] = counter.most_common(1)[0][0]
        return pred_res

In [ ]:
lr = LogisticRegression()
stac = StackingClassifier(classifiers=(rfc, mlpc, knc), meta_classifier=lr)
stac.fit(new_X_train, y_train)
y_pre_rfc = stac.predict(new_X_test)
print('正确标签：', y_test)
print('预测结果：', y_pre_rfc)

print('训练集分数：', rfc.score(new_X_train, y_train))
print('测试集分数：', rfc.score(new_X_test, y_test))
# 混淆矩阵
conf_mat = confusion_matrix(y_test, y_pre_rfc)
plt.matshow(conf_mat, cmap=plt.cm.Blues)
for (i, j), z in np.ndenumerate(conf_mat):
    plt.text(j, i, '{0:d}'.format(z), ha='center', va='center')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

print('混淆矩阵：')
print(conf_mat)


# 分类指标文本报告（精确率、召回率、F1值等）
print('分类指标报告：')
print(classification_report(y_test, y_pre_rfc))


# 画图展示训练结果
fig = plt.figure()
ax = fig.add_subplot(111)
f2 = ax.scatter(list(range(len(X_test))), y_pre_rfc, marker='o')
f1 = ax.scatter(list(range(len(X_test))), y_test, marker='*')
plt.legend(handles=[f1, f2], labels=['True', 'Prediction'])
plt.show()

## 绘制ROC曲线

In [ ]:
## 输出高清图像
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import platform
## 图像显示中文的问题，需要判断系统是windows还是苹果的
import matplotlib
import platform
sys_platform = platform.platform().lower()
if "windows" in sys_platform:
    font = {
    "family": "Microsoft YaHei"
    }
    matplotlib.rc("font", **font)
else:
    font = {
    "family": "Arial Unicode MS"
    }
    matplotlib.rc("font", **font)




In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
# 可视化在验证集上的Roc曲线
plt.figure(figsize=(10,8))
ax = plt.gca()
plot_roc_curve(rfc, new_X_test, y_test, ax=ax)
plot_roc_curve(mlpc, new_X_test, y_test, ax=ax)
plot_roc_curve(knc, new_X_test, y_test, ax=ax)
# stacking模型
fpr,tpr, thresholds = roc_curve(y_test, stac.decision_function(new_X_test))
stack_auc = roc_auc_score(y_test, stac.decision_function(new_X_test))
plt.plot(fpr, tpr, label="Stacking(AUC={:.2f})".format(stack_auc))
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['axes.unicode_minus'] = False
ax.plot([0, 1], [0, 1], 'k--')
plt.xlabel("假正率")
plt.ylabel("真正率")
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title("ROC曲线")
plt.legend()
plt.show()

## 保存训练好的模型

# 回归问题

## Libs

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso,Ridge, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from mlxtend.regressor import StackingRegressor
import xgboost as xgb

## 特征过程

In [ ]:
x_label = ["m1", "m2", "m3", "m4", "m5", "m6", "m7", "m8", "m9", "k1", "k2", "k3", "k4", "k5", "k6", "k7", "k8", "k9", "c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9", "sa1", "sa2", "sa3", "sa4", "sa5", "sa6", "sa7", "sa8", "sa9", "sa10", "sa11", "sa12", "sa13", "sa14", "sa15", "sa16", "sa17", "sa18", "sa19", "sa20", "sa21", "sa22", "sa23", "sa24", "sa25", "sa26", "sa27", "sa28", "sa29", "sa30", "sv1", "sv2", "sv3", "sv4", "sv5", "sv6", "sv7", "sv8", "sv9", "sv10", "sv11", "sv12", "sv13", "sv14", "sv15", "sv16", "sv17", "sv18", "sv19", "sv20", "sv21", "sv22", "sv23", "sv24", "sv25", "sv26", "sv27", "sv28", "sv29", "sv30", "sd1", "sd2", "sd3", "sd4", "sd5", "sd6", "sd7", "sd8", "sd9", "sd10", "sd11", "sd12", "sd13", "sd14", "sd15", "sd16", "sd17", "sd18", "sd19", "sd20", "sd21", "sd22", "sd23", "sd24", "sd25", "sd26", "sd27", "sd28", "sd29", "sd30", "sa_max", "sa_avg", "sv_max", "sv_avg", "sd_max", "sd_avg", "pga", "pgv", "pgd", "epa", "epv", "epd", "pa", "pv", "pd", "ic"]
y_label = ["drift_max", "drift_avg", "a_max", "a_avg", "T_1"]
df_ok_origin = df[df.ok==True]
df_ok = df_ok_origin.copy()
df_ok.describe()
X = df_ok.loc[:, x_label].values
y = df_ok.loc[:, y_label].values



In [ ]:
# df_ok = df_ok_origin.copy()
# from scipy import stats
# from scipy.stats import norm
# import seaborn as sns
# #Normal Distribution of Sales Price
# mu, sigma = norm.fit(df_ok[yy_label])
# print("Mu : {:.2f}\nSigma : {:.2f}".format(mu,sigma))
#
# #Visualization
# sns.distplot(df_ok[yy_label],fit=norm)
# plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\Sigma=$ {:.2f})'.format(mu,sigma)],loc = 'best')
# plt.xlabel('SalePrice Distribution')
# plt.ylabel('Frequency')
#
# fig = plt.figure()
# res = stats.probplot(df_ok[yy_label],plot=plt)
# plt.show()

In [ ]:
# df_ok[yy_label] = np.log(df_ok[yy_label])
# # print(df_ok[yy_label])
# #Normal Distribution of New Sales Price
# mu, sigma = norm.fit(df_ok[yy_label])
# print("Mu : {:.2f}\nSigma : {:.2f}".format(mu,sigma))
#
# #Visualization
# sns.distplot(df_ok[yy_label],fit=norm)
# plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\Sigma=$ {:.2f})'.format(mu,sigma)],loc = 'best')
# plt.xlabel('SalePrice Distribution')
# plt.ylabel('Frequency')
# fig = plt.figure()
# res = stats.probplot(df_ok[yy_label], plot=plt)
# plt.show()

In [ ]:
from sklearn.model_selection  import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2, shuffle=True)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
y_train_theta_max = y_train[:, 0]
y_test_theta_max = y_test[:, 0]
y_train_theta_avg = y_train[:, 1]
y_test_theta_avg = y_test[:, 1]
y_train_a_max = y_train[:, 2]
y_test_a_max = y_test[:, 2]
y_train_a_avg = y_train[:, 3]
y_test_a_avg = y_test[:, 3]
y_train_T1 = y_train[:, 4]
y_test_T1 = y_test[:, 4]

In [ ]:
# 特征标准化，采用最大最小值标准化，转化后的值范围（0,1）
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
new_X_train = X_train
new_X_test = X_test
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
# normalizer = Normalizer(copy=True, norm='l2').fit(new_X_train)
# new_X_train = normalizer.transform(new_X_train)
# new_X_test = normalizer.transform(new_X_test)
ss = StandardScaler().fit(new_X_train)
new_X_train = ss.transform(new_X_train)
new_X_test = ss.transform(new_X_test)

new_X_T1_train = new_X_train[:, 0: 27]
new_X_T1_test = new_X_test[:, 0: 27]

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

#Validation function
n_folds = 5

def rmsle_cv(model, X_train, y_train):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train)
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)


import matplotlib.pyplot as plt
def plot_regression_results(ax, y_true, y_pred, title, scores, elapsed_time):
    """预测目标与真实目标的散点图。"""
    ax.plot([y_true.min(), y_true.max()],
            [y_true.min(), y_true.max()],'--r', linewidth=2)
    ax.scatter(y_true, y_pred, alpha=0.2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    ax.set_xlim([y_true.min(), y_true.max()])
    ax.set_ylim([y_true.min(), y_true.max()])
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    extra = plt.Rectangle((0, 0), 0, 0, fc="w", fill=False,
                          edgecolor='none', linewidth=0)
    ax.legend([extra], [scores], loc='upper left')
    title = title + '\n Evaluation in {:.2f} seconds'.format(elapsed_time)
    ax.set_title(title)

## 基础模型

* RandomForest Regressor

In [ ]:
rfr_theta_max = RandomForestRegressor()
rfr_theta_avg = RandomForestRegressor()
rfr_a_max = RandomForestRegressor()
rfr_a_avg = RandomForestRegressor()
# model_rf.fit(new_X_train,y_train)
# y_pred_rf = model_rf.predict(new_X_train)
# score_rf = np.sqrt(mean_squared_error(y_train, y_pred_rf))
# print("RandomForest Score(On Training DataSet)  :",score_rf)
#
# y_pred_rf_test = model_rf.predict(new_X_test)

* LASSO Regression

In [ ]:
lasso_theta_max = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
lasso_theta_avg = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
lasso_a_max = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
lasso_a_avg = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

* Elastic Net Regression

In [ ]:
ENet_theta_max = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
ENet_theta_avg = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
ENet_a_max = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
ENet_a_avg = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

* Kernel Ridge Regression :

In [ ]:
KRR_theta_max = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
KRR_theta_avg = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
KRR_a_max = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
KRR_a_avg = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

* Gradient Boosting Regression :

In [ ]:
GBoost_theta_max = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)
GBoost_theta_avg = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)
GBoost_a_max = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)
GBoost_a_avg = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)
GBoost_T1 = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                         max_depth=4, max_features='sqrt',
                                         min_samples_leaf=15, min_samples_split=10,
                                         loss='huber', random_state=5)

* XGBoost :

In [ ]:
model_xgb_theta_max = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,
                             learning_rate=0.05, max_depth=3,
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_xgb_theta_avg = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,
                             learning_rate=0.05, max_depth=3,
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_xgb_a_max = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,
                             learning_rate=0.05, max_depth=3,
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_xgb_a_avg = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,
                             learning_rate=0.05, max_depth=3,
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

## Base models scores

In [ ]:
score = rmsle_cv(rfr_theta_max, new_X_train, y_train_theta_max)
print("\nRfr_theta_max score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(rfr_theta_avg, new_X_train, y_train_theta_avg)
print("\nRfr_theta_avg score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(rfr_a_max, new_X_train, y_train_a_max)
print("\nRfr_a_max score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(rfr_a_avg, new_X_train, y_train_a_avg)
print("\nRfr_a_avg score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(lasso_theta_max, new_X_train, y_train_theta_max)
print("\nLasso_theta_max score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(lasso_theta_avg, new_X_train, y_train_theta_avg)
print("\nLasso_theta_avg score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(lasso_a_max, new_X_train, y_train_a_max)
print("\nLasso_a_max score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(lasso_a_avg, new_X_train, y_train_a_avg)
print("\nLasso_a_avg score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(ENet_theta_max, new_X_train, y_train_theta_max)
print("\nENet_theta_max score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(ENet_theta_avg, new_X_train, y_train_theta_avg)
print("\nENet_theta_avg score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(ENet_a_max, new_X_train, y_train_a_max)
print("\nENet_a_max score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(ENet_a_avg, new_X_train, y_train_a_avg)
print("\nENet_a_avg score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(KRR_theta_max, new_X_train, y_train_theta_max)
print("\nKernel Ridge_theta_max score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(KRR_theta_avg, new_X_train, y_train_theta_avg)
print("\nKernel Ridge_theta_avg score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(KRR_a_max, new_X_train, y_train_a_max)
print("\nKernel Ridge_a_max score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(KRR_a_avg, new_X_train, y_train_a_avg)
print("\nKernel Ridge_a_avg score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(GBoost_theta_max, new_X_train, y_train_theta_max)
print("\nGradient Boosting_theta_max score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(GBoost_theta_avg, new_X_train, y_train_theta_avg)
print("\nGradient Boosting_theta_avg score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(GBoost_a_max, new_X_train, y_train_a_max)
print("\nGradient Boosting_a_max score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(GBoost_a_avg, new_X_train, y_train_a_avg)
print("\nGradient Boosting_a_avg score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_xgb_theta_max, new_X_train, y_train_theta_max)
print("\nXgboost_theta_max score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv(model_xgb_theta_avg, new_X_train, y_train_theta_avg)
print("\nXgboost_theta_avg score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(model_xgb_a_max, new_X_train, y_train_a_max)
print("\nXgboost_a_max score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(model_xgb_a_avg, new_X_train, y_train_a_avg)
print("\nXgboost_a_avg score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
stacked_averaged_models_theta_max = StackingRegressor(regressors = (ENet_theta_max, rfr_theta_max, KRR_theta_max),
                                                 meta_regressor = lasso_theta_max)

score = rmsle_cv(stacked_averaged_models_theta_max, new_X_train, y_train_theta_max)
print("Stacking Averaged models theta_max score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

stacked_averaged_models_theta_avg = StackingRegressor(regressors = (ENet_theta_avg, rfr_theta_avg, KRR_theta_avg),
                                                 meta_regressor = lasso_theta_avg)

score = rmsle_cv(stacked_averaged_models_theta_avg, new_X_train, y_train_theta_avg)
print("Stacking Averaged models theta_avg score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

stacked_averaged_models_a_max = StackingRegressor(regressors = (ENet_a_max, rfr_a_max, KRR_a_max),
                                                 meta_regressor = lasso_a_max)

score = rmsle_cv(stacked_averaged_models_a_max, new_X_train, y_train_a_max)
print("Stacking Averaged models a_max score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

stacked_averaged_models_a_avg = StackingRegressor(regressors = (ENet_a_avg, rfr_a_avg, KRR_a_avg),
                                                 meta_regressor = lasso_a_avg)

score = rmsle_cv(stacked_averaged_models_a_avg, new_X_train, y_train_a_avg)
print("Stacking Averaged models a_avg score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [ ]:
import matplotlib.pyplot as plt
def plot_regression_results(ax, y_true, y_pred, title, scores):
    """预测目标与真实目标的散点图。"""
    ax.plot([y_true.min(), y_true.max()],
            [y_true.min(), y_true.max()],'--r', linewidth=2)
    ax.scatter(y_true, y_pred, alpha=0.2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    ax.set_xlim([y_true.min(), y_true.max()])
    ax.set_ylim([y_true.min(), y_true.max()])
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    extra = plt.Rectangle((0, 0), 0, 0, fc="w", fill=False,
                          edgecolor='none', linewidth=0)
    ax.legend([extra], [scores], loc='upper left')
    ax.set_title(title)


def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
select_model_theta_max = GBoost_theta_max
select_model_theta_avg = GBoost_theta_avg
select_model_a_max = GBoost_a_max
select_model_a_avg = GBoost_a_avg
select_model_T1 = GBoost_T1

select_model_theta_max.fit(new_X_train, y_train_theta_max)
select_model_theta_avg.fit(new_X_train, y_train_theta_avg)
select_model_a_max.fit(new_X_train, y_train_a_max)
select_model_a_avg.fit(new_X_train, y_train_a_avg)
select_model_T1.fit(new_X_T1_train, y_train_T1)

train_pred_theta_max = select_model_theta_max.predict(new_X_train)
pred_theta_max = select_model_theta_max.predict(new_X_test)
print(rmsle(y_train_theta_max, train_pred_theta_max))

train_pred_theta_avg = select_model_theta_avg.predict(new_X_train)
pred_theta_avg = select_model_theta_avg.predict(new_X_test)
print(rmsle(y_train_theta_avg, train_pred_theta_avg))

train_pred_a_max = select_model_a_max.predict(new_X_train)
pred_a_max = select_model_a_max.predict(new_X_test)
print(rmsle(y_train_a_max, train_pred_a_max))

train_pred_a_avg = select_model_a_avg.predict(new_X_train)
pred_a_avg = select_model_a_avg.predict(new_X_test)
print(rmsle(y_train_a_avg, train_pred_a_avg))

train_pred_T1 = select_model_T1.predict(new_X_T1_train)
pred_T1 = select_model_T1.predict(new_X_T1_test)
print(rmsle(y_train_T1, train_pred_T1))

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
r2_theta_max = r2_score(y_test_theta_max, pred_theta_max)
mse_theta_max = mean_squared_error(y_test_theta_max, pred_theta_max)
mape_theta_max = mean_absolute_percentage_error(y_test_theta_max, pred_theta_max)

plt.figure(figsize=(10,8))
ax = plt.gca()
plot_regression_results(
        ax, y_test_theta_max, pred_theta_max,
        "Stacking Regressor",
        (r'$R^2={:.2f}$' + '\n' + r'$MAPE={:.2f}$' + '\n' + r'$MSE={:.2f}$')
        .format(r2_theta_max, mse_theta_max, mape_theta_max))


r2_theta_avg = r2_score(y_test_theta_avg, pred_theta_avg)
mse_theta_avg = mean_squared_error(y_test_theta_avg, pred_theta_avg)
mape_theta_avg = mean_absolute_percentage_error(y_test_theta_avg, pred_theta_avg)

plt.figure(figsize=(10,8))
ax = plt.gca()
plot_regression_results(
        ax, y_test_theta_avg, pred_theta_avg,
        "Stacking Regressor",
        (r'$R^2={:.2f}$' + '\n' + r'$MAPE={:.2f}$' + '\n' + r'$MSE={:.2f}$')
        .format(r2_theta_avg, mse_theta_avg, mape_theta_avg))


r2_a_max = r2_score(y_test_a_max, pred_a_max)
mse_a_max = mean_squared_error(y_test_a_max, pred_a_max)
mape_a_max = mean_absolute_percentage_error(y_test_a_max, pred_a_max)

plt.figure(figsize=(10,8))
ax = plt.gca()
plot_regression_results(
        ax, y_test_a_max, pred_a_max,
        "Stacking Regressor",
        (r'$R^2={:.2f}$' + '\n' + r'$MAPE={:.2f}$' + '\n' + r'$MSE={:.2f}$')
        .format(r2_a_max, mse_a_max, mape_a_max))


r2_a_avg = r2_score(y_test_a_avg, pred_a_avg)
mse_a_avg = mean_squared_error(y_test_a_avg, pred_a_avg)
mape_a_avg = mean_absolute_percentage_error(y_test_a_avg, pred_a_avg)

plt.figure(figsize=(10,8))
ax = plt.gca()
plot_regression_results(
        ax, y_test_a_avg, pred_a_avg,
        "Stacking Regressor",
        (r'$R^2={:.2f}$' + '\n' + r'$MAPE={:.2f}$' + '\n' + r'$MSE={:.2f}$')
        .format(r2_a_avg, mse_a_avg, mape_a_avg))


r2_T1 = r2_score(y_test_T1, pred_T1)
mse_T1 = mean_squared_error(y_test_T1, pred_T1)
mape_T1 = mean_absolute_percentage_error(y_test_T1, pred_T1)

plt.figure(figsize=(10,8))
ax = plt.gca()
plot_regression_results(
        ax, y_test_T1, pred_T1,
        "Stacking Regressor",
        (r'$R^2={:.2f}$' + '\n' + r'$MAPE={:.2f}$' + '\n' + r'$MSE={:.2f}$')
        .format(r2_T1, mse_T1, mape_T1))